In [8]:
import numpy as np

num_entities = 10

# h,r -> all t:w
scores = np.random.rand(num_entities)
print(scores)

[0.50381691 0.49621979 0.3216449  0.06411328 0.56871843 0.06535065
 0.11199909 0.73430975 0.17729558 0.82850504]


In [9]:
scores.argsort()[::-1]

array([9, 7, 4, 0, 1, 2, 8, 6, 5, 3])

In [10]:
scores.argsort()[::-1].argsort() + 1

array([ 4,  5,  6, 10,  3,  9,  8,  2,  7,  1])